In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon


In [ ]:
import osmnx
import openrouteservice
porto_area = osmnx.geocode_to_gdf('Porto, Portugal')

porto_area.plot()

center_porto = porto_area['geometry'].centroid

import folium
from folium.map import *
from openrouteservice import client

# openrouteservice api key
api_key = "5b3ce3597851110001cf6248516d2c7fa9b240a1989c451ecb3b2520"  
clnt = client.Client(key=api_key)

m = folium.Map(location=[41.16181,-8.62057], zoom_start=14)
m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TRAFFIC_DATA_PATH = [
    'traffic_flow_2018.csv',
    'traffic_flow_2019.csv'
]


def load_data(data_paths, date_col, value_cols, agg_operation = 'sum'):
    concat_data = []

    for file in data_paths:
        print("Reading file ", file)
        df = pd.read_csv('/content/drive/MyDrive/WDL/' + file, encoding='latin-1')
        
        # print(df.head())

        df[date_col] = pd.to_datetime(df[date_col])
        df_resampled = df.set_index(date_col).groupby('entity_id')[value_cols].resample('H').agg(agg_operation).reset_index()
        concat_data.append(df_resampled)

    return pd.concat(concat_data)

date_col = 'dateobservedfrom'
value_cols = ['intensity']
files_name = TRAFFIC_DATA_PATH
traffic_data = load_data(files_name, date_col, value_cols, agg_operation = 'sum')
traffic_data.rename(columns={'dateobservedfrom': 'dateobserved'}, inplace=True)

In [ ]:
data_entities = pd.read_csv('/content/drive/MyDrive/WDL/data_entities.csv', sep=',', encoding='latin-1', error_bad_lines=False)
df_resampled = traffic_data.merge(data_entities, on='entity_id')
df_resampled.latitude = df_resampled.latitude.astype(float)
df_resampled.longitude = df_resampled.longitude.astype(float)

In [ ]:
import pandas as pd
import folium



df_resampled.pickup_datetime = pd.to_datetime(df_resampled.dateobserved, format='%Y-%m-%d %H:%M:%S')
df_resampled['month'] = df_resampled.dateobserved.apply(lambda x: x.month)
df_resampled['week'] = df_resampled.dateobserved.apply(lambda x: x.week)
df_resampled['day'] = df_resampled.dateobserved.apply(lambda x: x.day)
df_resampled['hour'] = df_resampled.dateobserved.apply(lambda x: x.hour)
df_resampled['year'] = df_resampled.dateobserved.apply(lambda x: x.year)

In [ ]:
#Exemplo
df_copy = df_resampled[df_resampled.year==2018]
df_copy_1 = df_copy[df_copy.month==2]
df_copy_2 = df_copy_1[df_copy_1.day==10]



df_hour_list = []
for hour in df_copy_2.hour.sort_values().unique():
    df_hour_list.append(df_copy_2.loc[df_copy.hour == hour, ['latitude', 'longitude', 'intensity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())

In [ ]:
from folium.plugins import HeatMapWithTime

HeatMapWithTime(df_hour_list, radius=1, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(m)
m